In [25]:
from modules.data_loading import load_train_data, load_test_data
from modules.data_preprocessing import clean_data, feature_engineering
import pandas as pd
import numpy as np
import ydata_profiling
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import re
from datetime import datetime

In [2]:
df=load_train_data()
df_cleaned = clean_data(df)

,ID,Prix,Un. rés.,Rev. brut. pot.,YearBuilt,Type de bâtiment,Dimensions du bâtiment,Éval. terrain,Superficie habitable,Éval. bâtiment,...,Foyers-Poêles,Équip./Serv.,Rénovations,Inclusions,Exclusions,Remarques - Courtier,Addenda,Nbre pièces,Nbre chambres (hors-sol + sous-sol),Nbre salles de bains + salles d'eau
0,1,1 000 000 $,3,46 200 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,NaN,254 400 $,"2 972,99 pc / 276,2 mc",755 700 $,...,NaN,"Ascenseur(s), Buanderie",NaN,NaN,Effets personnels des locataires,Ne manquez pas cette incroyable opportunité de...,"Bienvenue à St-Henri, un quartier dynamique co...",8.0,3.0,1+0
1,2,725 000 $,3,37 200 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X p irr / 7,62 X m irr",254 400 $,"2 895 pc / 268,95 mc",539 100 $,...,NaN,NaN,NaN,NaN,NaN,Triplex situé au coeur du très convoité secteu...,CARACTÉRISTIQUES :+Stationnement extérieur+Rev...,5.0,3.0,1+0
2,3,725 000 $,3,37 500 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X p irr / 7,62 X m irr",254 400 $,"2 897,64 pc / 269,2 mc",513 600 $,...,NaN,NaN,NaN,NaN,NaN,Triplex situé au coeur du très convoité secteu...,"LOCALISATION+ Emplacement convoité, près de to...",5.0,3.0,1+0
3,4,942 000 $,3,54 660 $,Triplex dans le quartier Saint-Henri/Petite Bo...,En rangée,"25 X 58,2 p irr / 7,62 X 17,73 m irr",254 400 $,NaN,939 400 $,...,NaN,Thermopompe murale,NaN,"Cuisinière, réfrigérateur, lave-vaisselle, lav...","Cuisinière, réfrigérateur, lave-vaisselle, lav...","Triplex dans Saint-Henri, un secteur très rech...","Les visites débuteront le jeudi 17 novembre, 2...",5.0,3.0,1+0
4,5,732 000 $,2,32 400 $,Duplex dans le quartier Saint-Paul/Émard const...,En rangée,"25 X 45 p / 7,62 X 13,72 m",149 400 $,"2 292 pc / 212,93 mc",274 100 $,...,NaN,NaN,NaN,"5933 : Fixtures, stores, rideaux et pôles, hot...",5933 : Tous les biens du locataires,Opportunité intéressante pour investisseurs ou...,"Voici le 5933-5935, un duplex situé sur une ru...",5.0,3.0,1+0


In [14]:
df.columns

Index(['ID', 'price', 'units', 'income', 'YearBuilt', 'build_type',
       'Dimensions du bâtiment', 'land_eval', 'Superficie habitable',
       'build_eval', 'Dimensions du terrain', 'Cert. de localisation',
       'Superficie du terrain', 'Plan d'eau', 'Piscine',
       'Stationnement (total)', 'Chauffage', 'Eau (accès)', 'Foyers-Poêles',
       'Équip./Serv.', 'renovations', 'inclusions', 'exclusions', 'remarks',
       'addenda', 'rooms', 'Nbre chambres (hors-sol + sous-sol)',
       'Nbre salles de bains + salles d'eau', 'year_built', 'living_area'],
      dtype='object')

In [3]:
# Column naming
df.rename(columns={
    'Prix': 'price',
    'Un. rés.': 'units',
    'Rev. brut. pot.': 'income',
    'Type de bâtiment':'build_type',
    'Éval. bâtiment': 'build_eval',
    'Éval. terrain': 'land_eval',
    'Remarques - Courtier': 'remarks',
    'Rénovations': 'renovations',
    'Inclusions': 'inclusions',
    'Exclusions': 'exclusions',
    'Addenda': 'addenda',
    'Nbre pièces': 'rooms'
}, inplace=True)

In [4]:
# Datatypes
for col in ['price', 'income', 'build_eval', 'land_eval']:
    df[col] = df[col].str.replace('[\$, ]', '', regex=True).fillna(0).astype(int)

In [5]:
# Remove outliers in income
df = df[df['income'] >= 2000]

In [6]:
print(str(df['YearBuilt'].iloc[0]))

Triplex dans le quartier Saint-Henri/Petite Bourgogne construit en 1900


In [7]:
# Extract year of construction
def extract_year(row):
    year_match = re.search(r'\b\d{4}\b', row)
    return int(year_match.group()) if year_match else None

# Extract year and create a new column
df['year_built'] = df['YearBuilt'].apply(extract_year)

# Drop rows where 'year_built' is None
df.dropna(subset=['year_built'], inplace=True)

# Convert 'year_built' to integer
df['year_built'] = df['year_built'].astype(int)
df['year_built'].unique()

array([1900, 1910, 1930, 1915, 1923, 1953, 1952, 1931, 1958, 1980, 1955,
       1956, 1954, 1885, 1951, 1993, 1949, 1928, 1965, 1957, 1962, 1875,
       1948, 1964, 1880, 1981, 1950, 1959, 1968, 1963, 1946, 1922, 1937,
       1941, 1967, 1920, 1927, 1903, 1925, 1960, 1961, 1944, 1985, 1929,
       1966, 1970, 2001, 1932, 1924, 1890, 1934, 1999, 1987, 1945, 1947,
       1909, 1943, 1914, 1972, 1988, 1942, 1926, 1996, 1971, 1917, 1936,
       2010, 1969, 1974, 1975, 1978, 1976, 1973, 1940, 1902, 1912, 1905,
       1933, 1911, 1907, 1870, 1939, 1935, 1984, 2019, 1991, 1986, 1977,
       2000, 1989, 1979, 2020, 2008, 2009, 2017, 2004, 1892, 1895, 1918,
       1908, 1921, 1990, 1983, 1992, 1916, 1919, 1982, 1997])

In [9]:
df.shape[0]

1779

In [10]:
# Filling missing value for living_area using building dimension
def extract_living_area(row):
    # Extract living area if available
    if pd.notna(row['Superficie habitable']):
        match = re.search(r'([\d\s,]+)', row['Superficie habitable'].split('/')[0])
        if match and match.group(1).strip():
            return float(match.group(1).replace(',', '.').replace(' ', ''))
    
    # If living area is not available, use dimensions to calculate it
    if pd.notna(row['Dimensions du bâtiment']):
        match = re.findall(r'(\d+,\d+|\d+)', row['Dimensions du bâtiment'].split('/')[0])
        if len(match) >= 2:
            return float(match[0].replace(',', '.')) * float(match[1].replace(',', '.'))
    
    return np.nan

# Create the 'living_area' column
df['living_area'] = df.apply(extract_living_area, axis=1)

In [13]:
df.shape[0]

1758

In [12]:
# Drop rows where 'living_area' is NaN
df.dropna(subset=['living_area'], inplace=True)

In [15]:
# Filling missing value for yard_area using land dimension
def extract_yard_area(row):
    # Extract yard area if available
    if pd.notna(row['Superficie du terrain']):
        match = re.search(r'([\d\s,]+)', row['Superficie du terrain'].split('/')[0])
        if match and match.group(1).strip():
            return float(match.group(1).replace(',', '.').replace(' ', ''))
    
    # If living area is not available, use dimensions to calculate it
    if pd.notna(row['Dimensions du terrain']):
        match = re.findall(r'(\d+,\d+|\d+)', row['Dimensions du terrain'].split('/')[0])
        if len(match) >= 2:
            return float(match[0].replace(',', '.')) * float(match[1].replace(',', '.'))
    
    return np.nan

# Create the 'living_area' column
df['yard_area'] = df.apply(extract_yard_area, axis=1)

In [20]:
df.shape[0]

1758

In [20]:
# Drop rows where 'yard_area' is NaN
df.dropna(subset=['yard_area'], inplace=True)
df.shape[0]

1758

In [28]:
# Extract certificate boolean and year (and overdue status)
def extract_certificate_info(row):
    has_certificate = 0
    year_certificate = 0
    due_certificate = 0
    
    if 'Oui' in row:
        has_certificate = 1
        match = re.search(r'\((\d{4})\)', row)
        if match:
            year_certificate = int(match.group(1))
            current_year = datetime.now().year
            if current_year - year_certificate > 10:
                due_certificate = 1
                
    return pd.Series([has_certificate, year_certificate, due_certificate], index=['has_certificate', 'year_certificate', 'due_certificate'])

# Create new columns
df[['has_certificate', 'year_certificate', 'due_certificate']] = df['Cert. de localisation'].apply(extract_certificate_info)

In [33]:
df['Plan d\'eau'].unique()

array([nan, 'Fleuve St-Laurent', 'Canal de Lachine', 'Canal Lachine',
       'ville', 'Rivière des Prairies', 'VILLE', 'municipal',
       'St Laurent', 'St-Lawrence River', 'Fleuve Saint-Laurent', 'Ville',
       'none', 'rue municipal', 'Municipalité', 'Riviere-des-Prairies',
       'FLEUVE ST-LAURENT', 'St Lawrence', 'City', 'louis veuillot',
       'Municipality', "Canal de l'aqueduc", 'St-Laurent River',
       'Lachine canal', 'St-Lawrence'], dtype=object)

In [41]:
# Standard names list
standard_names = ['Fleuve St-Laurent', 'Canal de Lachine', 'Rivière des Prairies', 'Louis Veuillot', 'Municipal', 'Ville']

# Mapping of similar or translated terms to standard names
mapping = {
    'fleuve st-laurent': 'Fleuve St-Laurent',
    'st-lawrence river': 'Fleuve St-Laurent',
    'st-laurent river': 'Fleuve St-Laurent',
    'st lawrence': 'Fleuve St-Laurent',
    'st-lawrence': 'Fleuve St-Laurent',
    'canal de lachine': 'Canal de Lachine',
    'canal lachine': 'Canal de Lachine',
    'canal de l\'aqueduc': 'Canal de Lachine',
    'Lachine canal': 'Canal de Lachine',
    'lachine canal': 'Canal de Lachine',
    'rivière des prairies': 'Rivière des Prairies',
    'rivière-des-prairies': 'Rivière des Prairies',
    'municipal': 'Municipal',
    'rue municipal': 'Municipal',
    'municipality': 'Municipal',
    'municipalité': 'Municipal',
    'city': 'Ville',
    'ville': 'Ville',
    'louis veuillot': 'Louis Veuillot'
}

def standardize_water(row):
    near_water = 0
    water_name = None
    
    if pd.notna(row):
        row_lower = row.lower()
        if row_lower != 'none':
            near_water = 1
            for key, value in mapping.items():
                if key in row_lower:
                    water_name = value
                    break
                
    return pd.Series([near_water, water_name], index=['near_water', 'water_name'])

# Create new columns
df[['near_water', 'water_name']] = df['Plan d\'eau'].apply(standardize_water)
# Cast to appropriate data types
df['near_water'] = df['near_water'].astype(int)

In [42]:
df['water_name'].unique()

array([nan, 'Fleuve St-Laurent', 'Canal de Lachine', 'Ville',
       'Rivière des Prairies', 'Municipal', 'Louis Veuillot'],
      dtype=object)

EDA Profiling

In [4]:
profile = df.profile_report()
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]